In [1]:
from hmr4d import os_chdir_to_proj_root

os_chdir_to_proj_root()

In [2]:
import torch
from tqdm import tqdm
from hmr4d.utils.smplx_utils import make_smplx
from hmr4d.utils.wis3d_utils import make_wis3d, add_motion_as_lines
from hmr4d.utils.geo_transform import apply_T_on_points, compute_T_ayfz2ay
from hmr4d.utils.camera_utils import get_camera_mat_zface, cartesian_to_spherical
import hmr4d.utils.matrix as matrix

hml3d_joints = torch.load("./inputs/hml3d/joints3d.pth")

/tmp/ipykernel_2645068/3271024438.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hml3d_joints = torch.load("./inputs/hml3d/joints3d.pth")


In [3]:
all_keys = list(hml3d_joints.keys())
idx = 300
print(hml3d_joints[all_keys[idx]]["name"])

DISTANCE = 4.5
MAX_ANGLE = 180
ELEVA_ANGLE = 5
N_VIEWS = 4


def get_j3d_T_w2c(k):
    joints_pos = hml3d_joints[k]["joints3d"]
    joints_pos = torch.tensor(joints_pos, dtype=torch.float32)
    T_ay2ayfz = compute_T_ayfz2ay(joints_pos[:1], inverse=True)[0]  # (4, 4)
    joints_pos = apply_T_on_points(joints_pos, T_ay2ayfz)
    root_pos = joints_pos[:, :1]  # (F, 1, 3)
    root_next = joints_pos[1:, :1]  # (F - 1, 1, 3)
    root_next = torch.cat([root_next, root_next[-1:]], dim=0)  # (F, 1, 3)
    joints_pos = torch.cat([joints_pos, root_next], dim=-2)  # (F, 23, 3)
    F, J, _ = joints_pos.shape

    distance = torch.ones((N_VIEWS,)) * DISTANCE
    max_angle = MAX_ANGLE / 180 * torch.pi
    start = torch.rand((1,)) * 2 * torch.pi
    interval = 2 * max_angle / N_VIEWS
    angle = [start + i * interval for i in range(N_VIEWS)]
    angle = torch.cat((angle), dim=-1)
    eleva_angle = torch.ones((N_VIEWS,)) * ELEVA_ANGLE / 180.0 * torch.pi
    cam_mat = get_camera_mat_zface(matrix.identity_mat()[None], distance, angle, eleva_angle)  # N, 4, 4
    cam_mat = cam_mat[None].expand(F, -1, -1, -1)  # (F, N, 4, 4)
    cam_mat = matrix.set_position(cam_mat, matrix.get_position(cam_mat) + root_pos)  # (F, N, 4, 4)
    T_w2c = torch.inverse(cam_mat)  # F, N, 4, 4

    return joints_pos, T_w2c


joints_pos, T_w2c = get_j3d_T_w2c(all_keys[idx])

inputs/amass/smplhg_raw/EKUT/300/PushBeamPedar_13_poses.npz


In [44]:
frame_ids = [20, 50, 80]

j3d_selected = joints_pos[frame_ids]
T_w2c_selected = T_w2c[frame_ids]


import matplotlib.pyplot as plt
from pathlib import Path
from hmr4d.utils.geo_transform import project_p2d, apply_T_on_points
from hmr4d.utils.wis3d_utils import KINEMATIC_CHAINS, get_const_colors, color_schemes, add_motion_as_lines
import numpy as np


def draw_2d_skeleton(j3d_frame1_cam, save_path=None, rgb_color=None):
    assert rgb_color is not None
    size = 400
    f = 900
    K = torch.tensor([[f, 0, size / 2], [0, f, size / 2], [0, 0, 1]])
    f2d_frame1_cam = project_p2d(j3d_frame1_cam, K, size)
    f2d_frame1_cam = f2d_frame1_cam.detach().cpu().numpy()[:22, :]

    plt.gca().set_facecolor((0, 0, 0, 0))
    # Plot lines
    kinematic_chain = KINEMATIC_CHAINS["smpl22"]
    for chain in kinematic_chain:
        for i in range(len(chain) - 1):
            x1, y1 = f2d_frame1_cam[chain[i]]
            x2, y2 = f2d_frame1_cam[chain[i + 1]]
            plt.plot([x1, x2], [y1, y2], "-", color=rgb_color, linewidth=7, solid_capstyle='round')

    # # plot circle
    # for i in range(f2d_frame1_cam.shape[0]):
    #     x, y = f2d_frame1_cam[i]
    #     # plt.plot(x, y, "o", markersize=10, markerfacecolor=[1, 1, 1], markeredgecolor=rgb_color, markeredgewidth=3)  # 空心圆，边缘有颜色
    #     plt.plot(x, y, ".", markersize=10, markerfacecolor=[1, 1, 1], markeredgecolor=rgb_color, markeredgewidth=3)  # 空心圆，边缘有颜色

    plt.xlim(0, size)
    plt.ylim(0, size)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().invert_yaxis()
    plt.gca().invert_xaxis()
    plt.gcf().set_dpi(300)  # 设置dpi高一些
    plt.axis("off")  # 关闭坐标轴
    if save_path is not None:
        plt.savefig(save_path, format="png", bbox_inches="tight", pad_inches=0, dpi=300, transparent=True)

    plt.clf()


# rgb_start_end = [[56, 239, 125], [17, 153, 142]]
# rgb_start_end = [[149, 215, 213], [74, 194, 154]]

rgb_start_end = [[125, 223, 215], [24, 96, 90]]
n_colors = 3
rgb_colors = [
    [int(rgb_start_end[0][i] + (rgb_start_end[1][i] - rgb_start_end[0][i]) * t / n_colors) for i in range(3)]
    for t in range(n_colors)
]

output_dir = Path("./outputs/figure/teaser_local_vel_v1")
output_dir.mkdir(parents=True, exist_ok=True)
png_paths = []
for idx in range(len(frame_ids)):
    for i in range(1):
        rgb_color = [n / 255 for n in rgb_colors[idx]]
        j3d_frame1_cam = apply_T_on_points(j3d_selected[idx], T_w2c_selected[idx][i])
        save_path = output_dir / f"2d_skeleton_frame{idx}_{i}.png"
        png_paths.append(save_path)
        draw_2d_skeleton(j3d_frame1_cam, save_path, rgb_color)

# Merge pngs in a row
import cv2
import numpy as np

pngs = [cv2.imread(png_path) for png_path in png_paths]
pngs = np.concatenate(pngs, axis=1)
cv2.imwrite(output_dir / "2d_skeleton.png", pngs)

True

<Figure size 1920x1440 with 0 Axes>